# VGGVox

In [1]:
import pandas as pd
import os
import numpy as np
import re

In [2]:
if not os.path.exists('VGGVox-PyTorch'):
    ! git clone https://github.com/Derpimort/VGGVox-PyTorch.git
    if not os.path.exists('VGGVox-PyTorch/dataset'):
        os.system('mkdir dataset')
    os.system('rm -r VGGVox-PyTorch/data')
    os.system('cp -r dataset/ VGGVox-PyTorch/dataset/' )
    os.chdir('VGGVox-PyTorch')
    ! pip install -r requirements.txt
    
else:
    if not os.path.exists('VGGVox-PyTorch/dataset'):
        os.system('mkdir dataset')
    os.system('rm -r VGGVox-PyTorch/data')
    os.system(' cp -r dataset/ VGGVox-PyTorch/dataset/' )
    os.chdir('VGGVox-PyTorch')
    ! pip install -r requirements.txt

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
import torch
from torch.utils.data import Subset, Dataset, DataLoader
from tqdm.auto import tqdm
from vggm import VGGM
import argparse
from train import AudioDataset, accuracy, ppdf, LOCAL_DATA_DIR, MODEL_DIR
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=VGGM(1251)
model.load_state_dict(torch.load(MODEL_DIR+"VGGM300_BEST_140_81.99.pth", map_location=device))
model.to(device)
model.eval()

### This is to extract an activation from one layer ...
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
pwd

In [ ]:
DATA_DIR = '/Users/isabel.rodriguez/Desktop/Isa/MUIT - Machine Learning/DLAS/project/VGGVox-PyTorch/dataset'

df = pd.read_csv('dataset/labels.csv')

df.insert(0, 'Path', 'dataset/')

#Datasets={"test":AudioDataset(df, DATA_DIR, is_train=False)}

# Dataloaders={i:DataLoader(Datasets[i], batch_size=1, shuffle=True, num_workers=2) for i in Datasets}

# embedding_arr = []

# for audio, labels in Dataloaders['test']:
#         audio = audio.to(device)
#         labels = labels.to(device)
#         model.classifier.fc7.register_forward_hook(get_activation('fc7'))
#         outputs = model(audio)

#         embedding_arr.append(activation['fc7'].cpu().numpy().reshape(-1))

# df_F['Embeddings'] = pd.Series(embedding_arr)

In [ ]:
df